In [1]:
%pip install openai langchain playwright beautifulsoup4 nest_asyncio tiktoken -- quiet

ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet
Note: you may need to restart the kernel to use updated packages.


In [2]:
!playwright install

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Insert your OpenAI API key here:

# import os
# os.environ['OPENAI_API_KEY'] = ''

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [4]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}

def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

In [5]:
import pprint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer


def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(docs,tags_to_extract=["a"])
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000,
                                                                    chunk_overlap=0)
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(
        schema=schema, content=splits[0].page_content
    )
    pprint.pprint(extracted_content)
    return extracted_content

urls = ['https://understandingdata.com/']
extracted_content = scrape_with_playwright(urls, schema=schema)

Extracting content with LLM
[{'news_article_summary': 'Projects Services Posts Contact data engineering',
  'news_article_title': 'About'},
 {'news_article_summary': 'React Development Python Programming Development '
                          'Prompt Engineering Web Scraping SaaS Applications',
  'news_article_title': 'Data Engineering'},
 {'news_article_summary': '',
  'news_article_title': 'Soft Skills for Programmers: Why They Matter and How '
                        'to Develop Them'},
 {'news_article_summary': '',
  'news_article_title': 'What Are Webhooks? And How Do They Relate to Data '
                        'Engineering?'},
 {'news_article_summary': '',
  'news_article_title': 'What is an API? And How Do They Relate to Data '
                        'Engineering?'}]


In [6]:
print("Extracted content:", extracted_content)

Extracted content: [{'news_article_title': 'About', 'news_article_summary': 'Projects Services Posts Contact data engineering'}, {'news_article_title': 'Data Engineering', 'news_article_summary': 'React Development Python Programming Development Prompt Engineering Web Scraping SaaS Applications'}, {'news_article_title': 'Soft Skills for Programmers: Why They Matter and How to Develop Them', 'news_article_summary': ''}, {'news_article_title': 'What Are Webhooks? And How Do They Relate to Data Engineering?', 'news_article_summary': ''}, {'news_article_title': 'What is an API? And How Do They Relate to Data Engineering?', 'news_article_summary': ''}]
